In [ ]:
import tkinter as tk
from tkinter import messagebox
import mysql.connector
import random
from faker import Faker
from datetime import datetime

# Initialize Faker instance
fake = Faker()

# Database Configuration
DB_CONFIG = {
    "host": "127.0.0.1",
    "user": "root",  # Replace with your MySQL username
    "password": "root",  # Replace with your MySQL password
    "database": "soil_management"
}

# Connect to MySQL server
def get_db_connection():
    return mysql.connector.connect(
        host=DB_CONFIG["host"],
        user=DB_CONFIG["user"],
        password=DB_CONFIG["password"],
        database=DB_CONFIG["database"]
    )

# Create soil_data table if it doesn't exist
def create_table():
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS soil_data (
        id INT AUTO_INCREMENT PRIMARY KEY,
        field_name VARCHAR(255),
        soil_type VARCHAR(255),
        ph_level FLOAT,
        moisture_content FLOAT,
        nitrogen_level FLOAT,
        phosphorus_level FLOAT,
        potassium_level FLOAT,
        sample_date DATE
    )
    """)
    conn.commit()
    conn.close()

# Function to generate random soil data
def generate_soil_data():
    soil_types = ["Clay", "Loam", "Sand", "Silt", "Peat", "Chalk", "Saline"]
    field_name = fake.company()  # Random field name
    soil_type = random.choice(soil_types)  # Random soil type
    ph_level = round(random.uniform(5.0, 8.5), 2)  # Random pH level between 5.0 and 8.5
    moisture_content = round(random.uniform(10.0, 50.0), 2)  # Random moisture content between 10-50%
    nitrogen_level = round(random.uniform(0.5, 2.5), 2)  # Random nitrogen level
    phosphorus_level = round(random.uniform(0.5, 2.0), 2)  # Random phosphorus level
    potassium_level = round(random.uniform(0.5, 2.0), 2)  # Random potassium level
    sample_date = fake.date_this_decade()  # Random date within this decade
    
    return (field_name, soil_type, ph_level, moisture_content, nitrogen_level, phosphorus_level, potassium_level, sample_date)

# Insert soil data into the database
def insert_soil_data():
    field_name = entry_field_name.get()
    soil_type = entry_soil_type.get()
    ph_level = entry_ph_level.get()
    moisture_content = entry_moisture_content.get()
    nitrogen_level = entry_nitrogen_level.get()
    phosphorus_level = entry_phosphorus_level.get()
    potassium_level = entry_potassium_level.get()
    sample_date = entry_sample_date.get()

    if not all([field_name, soil_type, ph_level, moisture_content, nitrogen_level, phosphorus_level, potassium_level, sample_date]):
        messagebox.showwarning("Input Error", "Please fill all fields.")
        return

    try:
        # Insert data into the database
        conn = get_db_connection()
        cursor = conn.cursor()
        cursor.execute("""
        INSERT INTO soil_data (field_name, soil_type, ph_level, moisture_content, nitrogen_level, phosphorus_level, potassium_level, sample_date)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """, (field_name, soil_type, float(ph_level), float(moisture_content), float(nitrogen_level), float(phosphorus_level), float(potassium_level), sample_date))
        conn.commit()
        conn.close()
        
        messagebox.showinfo("Success", "Data inserted successfully!")
    except Exception as e:
        messagebox.showerror("Error", str(e))

# View soil data from the database
def view_soil_data():
    try:
        conn = get_db_connection()
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM soil_data")
        rows = cursor.fetchall()
        conn.close()

        # Display data in the text box
        result_text.delete(1.0, tk.END)  # Clear previous content
        for row in rows:
            result_text.insert(tk.END, f"{row}\n")
    except Exception as e:
        messagebox.showerror("Error", str(e))

# Create main window
window = tk.Tk()
window.title("Soil Management System")

# Create input labels and fields
label_field_name = tk.Label(window, text="Field Name:")
label_field_name.pack()
entry_field_name = tk.Entry(window)
entry_field_name.pack()

label_soil_type = tk.Label(window, text="Soil Type:")
label_soil_type.pack()
entry_soil_type = tk.Entry(window)
entry_soil_type.pack()

label_ph_level = tk.Label(window, text="pH Level:")
label_ph_level.pack()
entry_ph_level = tk.Entry(window)
entry_ph_level.pack()

label_moisture_content = tk.Label(window, text="Moisture Content (%):")
label_moisture_content.pack()
entry_moisture_content = tk.Entry(window)
entry_moisture_content.pack()

label_nitrogen_level = tk.Label(window, text="Nitrogen Level:")
label_nitrogen_level.pack()
entry_nitrogen_level = tk.Entry(window)
entry_nitrogen_level.pack()

label_phosphorus_level = tk.Label(window, text="Phosphorus Level:")
label_phosphorus_level.pack()
entry_phosphorus_level = tk.Entry(window)
entry_phosphorus_level.pack()

label_potassium_level = tk.Label(window, text="Potassium Level:")
label_potassium_level.pack()
entry_potassium_level = tk.Entry(window)
entry_potassium_level.pack()

label_sample_date = tk.Label(window, text="Sample Date (YYYY-MM-DD):")
label_sample_date.pack()
entry_sample_date = tk.Entry(window)
entry_sample_date.pack()

# Buttons for inserting and viewing data
button_insert = tk.Button(window, text="Insert Soil Data", command=insert_soil_data)
button_insert.pack()

button_view = tk.Button(window, text="View Soil Data", command=view_soil_data)
button_view.pack()

# Textbox to display data
result_text = tk.Text(window, width=100, height=10)
result_text.pack()

# Initialize the table when the program starts
create_table()

# Run the Tkinter event loop
window.mainloop()
